<a href="https://colab.research.google.com/github/rahafalsulami/Face-Generation/blob/main/CIFAR-10/new_training_for_SD2GAN_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from keras.models import load_model
from keras.models import model_from_json
json_file = open('/content/Siamese_net_noise_Cifar-10.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
sn_model = model_from_json(loaded_model_json)
sn_model.load_weights("/content/Siamese_net_noise_Cifar-10.h5")
print("Loaded model from disk")

Loaded model from disk


In [6]:
# example of a dcgan on cifar10
from numpy import expand_dims
import numpy as np
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.cifar10 import load_data
# from tf.keras.optimizers import Adam tf.keras.optimizers.Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
import tensorflow as tf

In [7]:
# define the standalone discriminator model
def define_discriminator(in_shape=(32,32,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 4x4 image
	n_nodes = 256 * 4 * 4
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((4, 4, 256)))
	# upsample to 8x8
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 16x16
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# output layer
	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

def define_discriminator2():
	model = Sequential()
	model.add(Dense(128, input_shape=(1,), activation=tf.nn.relu))
	model.add(Dense(1, activation='sigmoid'))

	opt = tf.keras.optimizers.Adam(lr=0.00009, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, d2_model, sn_model, latent_dim):
  # make weights in the discriminator not trainable
  d_model.trainable = False
  d2_model.trainable = False
  sn_model.trainable = False

  z = tf.keras.layers.Input(shape=(latent_dim,))
  g_img = g_model(z)
  imgs1 = tf.keras.layers.Input((32,32,3))
  imgs2 = tf.keras.layers.Input((32,32,3))

  similarity = sn_model([imgs1, imgs2])
  similarity_validity = d2_model(similarity)
  validity = d_model(g_img)

  model = tf.keras.models.Model([z, [imgs1, imgs2]], [validity, similarity_validity])

  opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model

In [8]:
# load and prepare cifar10 training images
def load_real_samples():
	# load cifar10 dataset
	(trainX, _), (_, _) = load_data()
	# convert from unsigned ints to floats
	X = trainX.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X

# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

# create and save a plot of generated images
def save_plot(examples, epoch, n=2):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()


# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, d2_model, sn_model, dataset, latent_dim, n_samples=150):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# print(X_real.shape[0])
	# rand_idx = np.random.randint(0, n_samples, int(n_samples/2))
	# rand_idx2 = np.array([i for i in range(n_samples) if i not in rand_idx])
	splited_batch = np.split(X_real, 2)
	# print(rand_idx.shape, rand_idx2.shape)
	batch1 = splited_batch[0]
	batch2 = splited_batch[1]
	sim_real = sn_model.predict([batch1, batch2])
	y_sim_real = ones((sim_real.shape[0], 1))
	# rand_idx = np.random.randint(0, x_fake.shape[0])
	# rand_idx2 = np.array([i for i in range(x_fake.shape[0]) if i not in rand_idx])
	splited_batch = np.split(x_fake, 2)
	# batch1 = x_fake[rand_idx]
	# batch2 = x_fake[rand_idx2]
	batch1 = splited_batch[0]
	batch2 = splited_batch[1]
	sim_fake = sn_model.predict([batch1, batch2])
	y_sim_fake = zeros((sim_real.shape[0], 1))
	_, acc2_real = d2_model.evaluate(sim_real, y_sim_real, verbose=0)
	_, acc2_fake = d2_model.evaluate(sim_fake, y_sim_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%% \t >D2 Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100, acc2_real*100, acc2_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

In [9]:
# train the generator and discriminator
def train(g_model, d_model, d2_model, sn_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)
  # manually enumerate epochs
  for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
      # get randomly selected 'real' samples
      X_real, y_real = generate_real_samples(dataset, half_batch)
      # update discriminator model weights
      d_loss1, _ = d_model.train_on_batch(X_real, y_real)
      splited_batch = np.split(X_real, 2)
      batch1 = splited_batch[0]
      batch2 = splited_batch[1]
      sim_real = sn_model.predict([batch1, batch2])
      y_sim_real = ones((sim_real.shape[0], 1))
      d2_loss1, _ = d2_model.train_on_batch(sim_real, y_sim_real)
      # generate 'fake' examples
      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      # update discriminator model weights
      d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
      splited_batch = np.split(X_fake, 2)
      batch1 = splited_batch[0]
      batch2 = splited_batch[1]
      sim_fake = sn_model.predict([batch1, batch2])
      y_sim_fake = zeros((sim_fake.shape[0], 1))
      d2_loss1, _ = d2_model.train_on_batch(sim_fake, y_sim_fake)
      # prepare points in latent space as input for the generator
      X_gan = generate_latent_points(latent_dim, n_batch)
      # print(X_gan.shape)
      X_gan_f, y_gan_f = generate_fake_samples(g_model, latent_dim, n_batch*2)
      splited_batch = np.split(X_gan_f, 2)
      batch1 = splited_batch[0]
      batch2 = splited_batch[1]
      # sim_gan = sn_model.predict([batch1, batch2])
      # y_sim_gan = ones((sim_gan.shape[0], 1))
      # create inverted labels for the fake samples
      y_gan = ones((n_batch, 1))
      # update the generator via the discriminator's error
      g_loss = gan_model.train_on_batch([X_gan, [batch1, batch2]], y_gan)
      # print(g_loss)
      # summarize loss on this batch
      print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
        (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss[0]))
      # evaluate the model performance, sometimes
      if (i+1) % 10 == 0:
        summarize_performance(i, g_model, d_model, d2_model, sn_model, dataset, latent_dim)

In [10]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
d2_model = define_discriminator2()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model, d2_model, sn_model, latent_dim)
dataset = load_real_samples()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [7]:
train(g_model, d_model, d2_model, sn_model, gan_model, dataset, latent_dim)

Streaming output truncated to the last 5000 lines.
>Accuracy real: 64%, fake: 96% 	 >D2 Accuracy real: 52%, fake: 71%
>190, 285/390, d1=0.383, d2=0.502 g=2.201
>Accuracy real: 69%, fake: 95% 	 >D2 Accuracy real: 44%, fake: 72%
>190, 286/390, d1=0.406, d2=0.540 g=2.389
>Accuracy real: 74%, fake: 95% 	 >D2 Accuracy real: 48%, fake: 79%
>190, 287/390, d1=0.371, d2=0.492 g=2.268
>Accuracy real: 72%, fake: 97% 	 >D2 Accuracy real: 48%, fake: 77%
>190, 288/390, d1=0.402, d2=0.382 g=2.233
>Accuracy real: 75%, fake: 92% 	 >D2 Accuracy real: 56%, fake: 76%
>190, 289/390, d1=0.363, d2=0.434 g=2.046
>Accuracy real: 79%, fake: 87% 	 >D2 Accuracy real: 41%, fake: 67%
>190, 290/390, d1=0.460, d2=0.460 g=2.100
>Accuracy real: 75%, fake: 87% 	 >D2 Accuracy real: 56%, fake: 73%
>190, 291/390, d1=0.483, d2=0.782 g=2.124
>Accuracy real: 78%, fake: 85% 	 >D2 Accuracy real: 44%, fake: 73%
>190, 292/390, d1=0.444, d2=0.463 g=2.176
>Accuracy real: 70%, fake: 96% 	 >D2 Accuracy real: 37%, fake: 76%
>190, 293/

In [23]:
generator = g_model
generator.load_weights("/content/generator_model_190.h5")
print("Loaded model from disk")

Loaded model from disk


In [24]:
import math
import tensorflow as tf
import time
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm
from numpy import expand_dims
from keras.models import load_model
from numpy.random import randn
from keras.models import model_from_json
from keras.datasets import mnist
import numpy as np
import random
from keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize
from keras.applications.inception_v3 import InceptionV3

# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# x_train = x_train / 127.5 - 1.
# x_test = x_test / 127.5 - 1.
# print(x_train.shape)

# json_file = open('/content/output/generator.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# generator = model_from_json(loaded_model_json)
# generator.load_weights("/content/output/generator.h5")
# print("Loaded model from disk")

def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return np.asarray(images_list)
 
 # calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

In [34]:
for i in range(12):
    imgs_num = 500
    idx = np.random.randint(0, dataset.shape[0], imgs_num)
    # define two collections of 500 images
    reals = dataset[idx]
    # reals = reals.reshape((imgs_num,32,32,3))
    noise = np.random.normal(0, 1, (imgs_num, 100))
    # Generate a batch of new images
    fakes = g_model.predict(noise)
    gen_imgs = fakes
    fakes = fakes.reshape((imgs_num,32,32,3))
    # resize images
    reals = scale_images(reals, (299,299,3))
    fakes = scale_images(fakes, (299,299,3))
    reals = preprocess_input(reals)
    fakes = preprocess_input(fakes)
    # fid between images1 and images2
    fid = calculate_fid(model, reals, fakes)
    print('FID: %.3f' % fid)

FID: 1.272
FID: 1.426
FID: 0.880
FID: 1.113
FID: 0.951
FID: 1.487
FID: 0.815
FID: 0.889
FID: 1.084
FID: 0.956
FID: 0.771
FID: 1.192


FID:


1.   0.895

2.   0.915
3.0.804
4.0.803
5.0.740






In [35]:
# calculate inception score for cifar-10 in Keras
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy.random import shuffle
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets import mnist
from skimage.transform import resize
from numpy import asarray

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train / 127.5 - 1.
x_test = x_test / 127.5 - 1.
print(x_train.shape)


# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

# assumes images have any shape and pixels in [0,255]
def calculate_inception_score(images, n_split=10, eps=1E-16):
	# load inception v3 model
	model = InceptionV3()
	# enumerate splits of images/predictions
	scores = list()
	n_part = floor(images.shape[0] / n_split)
	for i in range(n_split):
		# retrieve images
		ix_start, ix_end = i * n_part, (i+1) * n_part
		subset = images[ix_start:ix_end]
		# convert from uint8 to float32
		subset = subset.astype('float32')
		# scale images to the required size
		subset = scale_images(subset, (299,299,3))
		# pre-process images, scale to [-1,1]
		# subset = preprocess_input(subset)
		# predict p(y|x)
		p_yx = model.predict(subset)
		# calculate p(y)
		p_y = expand_dims(p_yx.mean(axis=0), 0)
		# calculate KL divergence using log probabilities
		kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
		# sum over classes
		sum_kl_d = kl_d.sum(axis=1)
		# average over images
		avg_kl_d = mean(sum_kl_d)
		# undo the log
		is_score = exp(avg_kl_d)
		# store
		scores.append(is_score)
	# average across images
	is_avg, is_std = mean(scores), std(scores)
	return is_avg, is_std

# load cifar10 images
# (images, _), (_, _) = mnist.load_data()
# shuffle images
images_reals = dataset[:1000]
shuffle(images_reals)
print('loaded', images_reals.shape)
# calculate inception score
is_avg, is_std = calculate_inception_score(images_reals)
print('score', is_avg, is_std)

(50000, 32, 32, 3)
loaded (1000, 32, 32, 3)
score 9.242806 0.8963166


In [41]:
# load cifar10 images
noise = np.random.normal(0, 1, (1000, 100))
# Generate a batch of new images
images = generator.predict(noise)
shuffle(images)
print('loaded', images.shape)
# calculate inception score
is_avg, is_std = calculate_inception_score(images)
print('score', is_avg, is_std)

loaded (1000, 32, 32, 3)
score 5.976144 0.38315547


1-5.9019713

2- 6.1320715

3- 6.6.052945

4- 6.458251

5-6.231183